<a href="https://colab.research.google.com/github/doukansurel/Retrieval-Augmented-Generation/blob/main/RAG_Pipeline_From_Scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install llama-index
!pip install rank-bm25 pymupdf
!pip install pypdf
!pip install openai
!pip install llama_hub

In [ ]:
import logging
import sys
import nest_asyncio

nest_asyncio.apply()
logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

from llama_index import (
    VectorStoreIndex,
    SimpleDirectoryReader,
    ServiceContext,
    LLMPredictor,
)
import os
from llama_index.llms import OpenAI

In [ ]:
os.environ["OPENAI_API_KEY"] = "<OPENAI_API_KEY>"
openai.api_key = os.environ["OPENAI_API_KEY"]

# Doküman Yükleme


In [ ]:
from llama_index import SimpleDirectoryReader
documents = SimpleDirectoryReader("/content/data/").load_data()

#Vector Index

In [ ]:
from llama_index import VectorStoreIndex, ServiceContext

service_context = ServiceContext.from_defaults(chunk_size=1024)
index = VectorStoreIndex.from_documents(
    documents, service_context=service_context)

In [ ]:
llm = OpenAI(model="gpt-3.5-turbo")

#Sorgu Oluşturma

In [ ]:
from llama_index import PromptTemplate

In [ ]:
query_str = ( "Kira bedeli nasıl hesaplanır ve kiralanan araç ile kaza yapıldığında kim sorumludur?")


In [ ]:
query_str = "Araç kiralama süresi en az ne kadardır ve araç kiralamak için hangi şartları yerine getirmek gerekir?"


In [ ]:
query_gen_prompt_str = (
    "Birden fazla arama sorgusunu temel alan, yardımsever bir yardımcısınız"
    "tek giriş sorgusu. Her satırda bir tane olmak üzere {num_queries} arama sorgusu oluşturun, "
    "aşağıdaki giriş sorgusuyla ilgili:\n"
    "Sorgu: {query}\n"
    "Sorgular:\n"
)
query_gen_prompt = PromptTemplate(query_gen_prompt_str)


In [ ]:
def generate_queries(llm, query_str: str, num_queries: int = 4):
    fmt_prompt = query_gen_prompt.format(
        num_queries=num_queries - 1, query=query_str
    )
    response = llm.complete(fmt_prompt)
    queries = response.text.split("\n")
    return queries

In [ ]:
queries = generate_queries(llm, query_str, num_queries=4)

In [ ]:
print(queries)

['1. Kira bedeli nasıl hesaplanır?', '2. Kiralanan araç ile kaza yapıldığında kim sorumludur?', '3. Kaza durumunda kiralanan araç için ne yapılmalıdır?']


# Her Sorgu için Vektör Araması Yapma

*   Liste öğesi
*   Liste öğesi



In [ ]:
from tqdm.asyncio import tqdm


async def run_queries(queries, retrievers):
    """Run queries against retrievers."""
    tasks = []
    for query in queries:
        for i, retriever in enumerate(retrievers):
            tasks.append(retriever.aretrieve(query))

    task_results = await tqdm.gather(*tasks)

    results_dict = {}
    for i, (query, query_result) in enumerate(zip(queries, task_results)):
        results_dict[(query, i)] = query_result

    return results_dict

In [ ]:
# get retrievers
from llama_index.retrievers import BM25Retriever


## vector retriever
vector_retriever = index.as_retriever(similarity_top_k=2)

## bm25 retriever
bm25_retriever = BM25Retriever.from_defaults(
    docstore=index.docstore, similarity_top_k=2
)

In [ ]:
results_dict = await run_queries(queries, [vector_retriever, bm25_retriever])


100%|██████████| 6/6 [00:00<00:00, 31.29it/s]


#Füzyon Performansı Ayarlama

In [ ]:
def fuse_results(results_dict, similarity_top_k: int = 2):
    """Fuse results."""
    k = 60.0  # `k` is a parameter used to control the impact of outlier rankings.
    fused_scores = {}
    text_to_node = {}

    # compute reciprocal rank scores
    for nodes_with_scores in results_dict.values():
        for rank, node_with_score in enumerate(
            sorted(
                nodes_with_scores, key=lambda x: x.score or 0.0, reverse=True
            )
        ):
            text = node_with_score.node.get_content()
            text_to_node[text] = node_with_score
            if text not in fused_scores:
                fused_scores[text] = 0.0
            fused_scores[text] += 1.0 / (rank + k)

    # sort results
    reranked_results = dict(
        sorted(fused_scores.items(), key=lambda x: x[1], reverse=True)
    )

    # adjust node scores
    reranked_nodes: List[NodeWithScore] = []
    for text, score in reranked_results.items():
        reranked_nodes.append(text_to_node[text])
        reranked_nodes[-1].score = score

    return reranked_nodes[:similarity_top_k]

In [ ]:
final_results = fuse_results(results_dict)


In [ ]:
from llama_index.response.notebook_utils import display_source_node

for n in final_results:
    display_source_node(n, source_length=500)

**Node ID:** 55b69de8-96e4-4847-9346-2e73e36dd898<br>**Similarity:** 0.016666666666666666<br>**Text:** 7/71.  
2.  
3.  
4.  
5.  
6.  
7.  
8.  istinaden kıdem ve ihbar tazminatı da dahil olmak üzere işçilik hak ve alacakları 
hesaplanarak personelin maaş aldığı banka hesabına yatırılacak ödeme yapılacak ve 
personelin şirketten herhangi bir hak ve alacağı kalmadığına dair ibraname 
düzenlenecektir.
6-İşten ayrılan personele 4857 sayılı İş Kanunu’nun 28.md istinaden işin çeşidi ve 
çalışma süresini gösterir çalışma belgesi düzenlenerek verilir.
PERSONELİN UYMASI GEREKEN GENEL HUSUSLAR
Madde 1...<br>

**Node ID:** 6177309a-e1a4-4eda-a390-eded30fec5c8<br>**Similarity:** 0.016666666666666666<br>**Text:** 3) SÖZLEŞMENİN KONUSU :  
İşbu sözleşmenin konusu, Hasta’ nın her türlü sağlık i le ilgili sorunlarının Hastane ’de ayakta 
ve/veya yatarak tüm teşhis ve tedavilerinin sağlanması, bu süreç esnasında yapılacak işlemler ile 
uygulama esaslarının belirlenmesinden ibarettir.   
 
4) HAK VE YÜKÜMLÜLÜKLER :  
 
4.1) Hastanenin Hak ve Yükümlülükleri  
4.1.a)  Hastane , hizmet almak isteyen Hasta’ nın sağlık hizmetler ini eksiksiz olarak yerine 
getirecektir.  
4.1.b) Hastane, hizmet talebinde buluna...<br>

In [ ]:
from llama_index import QueryBundle
from llama_index.retrievers import BaseRetriever
from typing import Any, List
from llama_index.schema import NodeWithScore


class FusionRetriever(BaseRetriever):
    """Ensemble retriever with fusion."""

    def __init__(
        self,
        llm,
        retrievers: List[BaseRetriever],
        similarity_top_k: int = 2,
    ) -> None:
        """Init params."""
        self._retrievers = retrievers
        self._similarity_top_k = similarity_top_k

    def _retrieve(self, query_bundle: QueryBundle) -> List[NodeWithScore]:
        """Retrieve."""
        queries = generate_queries(llm, query_str, num_queries=4)
        results = run_queries(queries, [vector_retriever, bm25_retriever])
        final_results = fuse_results(
            results_dict, similarity_top_k=self._similarity_top_k
        )

        return final_results

In [ ]:
from llama_index.query_engine import RetrieverQueryEngine

fusion_retriever = FusionRetriever(
    llm, [vector_retriever, bm25_retriever], similarity_top_k=2
)

query_engine = RetrieverQueryEngine(fusion_retriever)

In [ ]:
response = query_engine.query(query_str)


/usr/local/lib/python3.10/dist-packages/llama_index/indices/base_retriever.py:34: RuntimeWarning: coroutine 'run_queries' was never awaited
  return self._retrieve(str_or_query_bundle)


In [ ]:
print(str(response))


Kira bedeli, araç kiralama sözleşmesinde belirtilen şartlara göre hesaplanır. Bu şartlar genellikle kiralama süresi, aracın marka ve modeli, kilometre sınırları gibi faktörlere dayanır. Kiralanan araç ile kaza yapıldığında, sorumluluk genellikle kiralayan şirkete aittir. Ancak, kiralama sözleşmesinde belirtilen sigorta koşulları ve sorumluluklar da dikkate alınmalıdır. Bu nedenle, kaza durumunda, kiralama sözleşmesinde belirtilen prosedürleri takip etmek ve sigorta şirketiyle iletişime geçmek önemlidir.


In [ ]:
print(str(response))


Araç kiralama süresi en az 2 aydır. Araç kiralamak için T.C. vatandaşı olmak veya yabancı ülke vatandaşı olup Türkiye'de oturma ve çalışma izni almasına engel teşkil edecek bir duruma sahip olmamak, işin gerektirdiği eğitim, bilgi, yetenek ve tecrübeye sahip olmak, yüz kızartıcı bir suçtan hüküm giymemiş olmak veya kamu haklarını kısıtlayıcı bir suçla mahkum edilmemiş olmak, askerlik hizmetini yapmış olmak veya askerlikle ilişiği bulunmamak, işin gerektirdiği sağlık durumuna sahip olmak ve İŞVEREN tarafından istenilen belgeleri temin etmek gerekmektedir.
